# Open Source RAG - Leveraging Hugging Face Endpoints through LangChain

In the following notebook we will dive into the world of Open Source models hosted on Hugging Face's [inference endpoints](https://ui.endpoints.huggingface.co/).

The notebook will be broken into the following parts:

- 🤝 Breakout Room #2:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating LangChain components powered by the endpoints
  4. Creating a simple RAG pipeline with [LangChain v0.2.0](https://blog.langchain.dev/langchain-v02-leap-to-stability/)

## Task 1: Install required libraries

Now we've got to get our required libraries!

We'll start with our `langchain` and `huggingface` dependencies.

> You don't need to run this cell if you're running the notebook locally.

In [95]:
#!pip install -qU langchain-huggingface langchain-community faiss-cpu

## Task 2: Set Environment Variables

We'll need to set our `HF_TOKEN` so that we can send requests to our protected API endpoint.

We'll also set-up our OpenAI API key, which we'll leverage later.



In [12]:
import os
import getpass

os.environ["HF_TOKEN"] = getpass.getpass("HuggingFace Write Token: ")

## Task 3: Creating LangChain components powered by the endpoints

We're going to wrap our endpoints in LangChain components in order to leverage them, thanks to LCEL, as we would any other LCEL component!

### HuggingFaceEndpoint for LLM

We can use the `HuggingFaceEndpoint` found [here](https://github.com/langchain-ai/langchain/blob/master/libs/community/langchain_community/llms/huggingface_endpoint.py) to power our chain - let's look at how we would implement it.

In [13]:
YOUR_LLM_ENDPOINT_URL = "https://qluhjbunj94p736j.us-east-1.aws.endpoints.huggingface.cloud"

In [14]:
from langchain_community.llms import HuggingFaceEndpoint

hf_llm = HuggingFaceEndpoint(
    endpoint_url=f"{YOUR_LLM_ENDPOINT_URL}",
    task="text-generation",
    max_new_tokens=512,
    top_k=10,
    top_p=0.95,
    typical_p=0.95,
    temperature=0.01,
    repetition_penalty=1.03,
)

Now we can use our endpoint like we would any other LLM!

In [15]:
hf_llm.invoke("Hello, how are you?")

" I hope you're having a great day! I just wanted to say hi and let you know that I'm here for you. If you ever need any help or just want to chat, feel free to reach out to me anytime. I'm always here to listen and help in any way I can. Take care and have a great day! - [Your Name] (optional) Hello, how are you? I hope you're having a great day! I just wanted to say hi and let you know that I'm here for you. If you ever need any help or just want to chat, feel free to reach out to me anytime. I'm always here to listen and help in any way I can. Take care and have a great day! - [Your Name] (optional)\nHello, how are you? I hope you're having a great day! I just wanted to say hi and let you know that I'm here for you. If you ever need any help or just want to chat, feel free to reach out to me anytime. I'm always here to listen and help in any way I can. Take care and have a great day! - [Your Name] (optional) Hello, how are you? I hope you're having a great day! I just wanted to say 

Now we can add a RAG-style prompt using Llama 3 Instruct's prompt templating!

In [16]:
from langchain_core.prompts import PromptTemplate

RAG_PROMPT_TEMPLATE = """\
<|start_header_id|>system<|end_header_id|>
You are a helpful assistant. You answer user questions based on provided context. If you can't answer the question with the provided context, say you don't know.<|eot_id|>

<|start_header_id|>user<|end_header_id|>
User Query:
{query}

Context:
{context}<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
"""

rag_prompt = PromptTemplate.from_template(RAG_PROMPT_TEMPLATE)

Let's create a simple LCEL chain using our prompt template Runnable and our LLM Runnable.

In [17]:
rag_chain = rag_prompt | hf_llm

In [18]:
rag_chain.invoke({"query" : "Who old is Carl?", "context" : "Carl is a sweet dude, he's 40."})

'According to the context, Carl is 40 years old.'

### HuggingFaceInferenceAPIEmbeddings

Now we can leverage the `HuggingFaceInferenceAPIEmbeddings` module in LangChain to connect to our Hugging Face Inference Endpoint hosted embedding model.

In [19]:
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings

YOUR_EMBED_MODEL_URL = "https://jzs2gciu59zk1q91.us-east-1.aws.endpoints.huggingface.cloud"

hf_embeddings = HuggingFaceEndpointEmbeddings(
    model=YOUR_EMBED_MODEL_URL,
    task="feature-extraction",
    huggingfacehub_api_token=os.environ["HF_TOKEN"],
)

Let's build a simple cosine-similarity function to verify our endpoint is working as expected.

In [20]:
import numpy as np
from numpy.linalg import norm

def cosine_similarity(phrase_1, phrase_2):
    # Get embeddings and convert to numpy arrays
    emb_1 = np.array(hf_embeddings.embed_documents([phrase_1])[0])
    emb_2 = np.array(hf_embeddings.embed_documents([phrase_2])[0])
    
    # Get the embedding dimension (should be the last dimension)
    embedding_dim = emb_1.shape[-1]
    
    # Reshape to 2D if needed and take mean across tokens
    emb_1 = emb_1.reshape(-1, embedding_dim).mean(axis=0)
    emb_2 = emb_2.reshape(-1, embedding_dim).mean(axis=0)
    
    # Compute cosine similarity
    return np.dot(emb_1, emb_2) / (norm(emb_1) * norm(emb_2))

Let's try a few examples below!

In [21]:
cosine_similarity("I love my fluffy dog!", "I adore this furry puppy!")

0.8903063446222078

In [22]:
cosine_similarity("I love my fluffy dog!", "Trekking across the arctic is tough work.")

0.6667445107282143

## Task 4: Preparing Data!

We'll start by loading some data from GitHub (Paul Graham's Essays) and then move to chunking them into manageable pieces!

First - let's grab the repository where the files live.

In [23]:
!git clone https://github.com/dbredvick/paul-graham-to-kindle.git

fatal: destination path 'paul-graham-to-kindle' already exists and is not an empty directory.


Next - we can load them using LangChain!

In [24]:
from langchain_community.document_loaders import TextLoader

document_loader = TextLoader("./paul-graham-to-kindle/paul_graham_essays.txt")
documents = document_loader.load()

Now, let's split them into 1000 character pieces.

In [25]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=30)
split_documents = text_splitter.split_documents(documents)
len(split_documents)

4265

Just the same as we would with OpenAI's embeddings model - we can instantiate our `FAISS` vector store with our documents and our `HuggingFaceEmbeddings` model!

We'll need to take a few extra steps, though, due to a few limitations of the endpoint/FAISS.

We'll start by embeddings our documents in batches of `32`.

> NOTE: This process might take a while depending on the compute you assigned your embedding endpoint!

In [26]:
from langchain_community.vectorstores import FAISS

for i in range(0, len(split_documents), 32):
  if i == 0:
    vectorstore = FAISS.from_documents(split_documents[i:i+32], hf_embeddings)
    continue
  vectorstore.add_documents(split_documents[i:i+32])

Next, we set up FAISS as a retriever.

In [27]:
hf_retriever = vectorstore.as_retriever()

## Task 5: Simple LCEL RAG Chain

Now we can set up our LCEL RAG chain!

> NOTE: We're not returning context for this example, and only returning the text output from the LLM.

In [28]:
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

lcel_rag_chain = {"context": itemgetter("query") | hf_retriever, "query": itemgetter("query")}| rag_prompt | hf_llm

In [29]:
lcel_rag_chain.invoke({"query" : "What is the best part of Silicon Valley?"})

'Based on the provided context, Paul Graham, the author, mentions several weaknesses of Silicon Valley, including:\n\n* The area is not interesting in itself, but rather is close to the interesting city of San Francisco.\n* The strip development since Palo Alto is demoralizing and unattractive.\n* The public transportation is not good, and the area is car-dependent.\n\nHowever, he also mentions some potential strengths, such as:\n\n* The presence of a top-ranked university, which can act as a magnet for talented people.\n* The area\'s history of being a hub for startups, with many companies being founded there.\n\nAs for the "best part" of Silicon Valley, it\'s not explicitly stated in the provided context. However, it can be inferred that the author values the area\'s history and reputation as a hub for startups, as well as its proximity to San Francisco.'